## Runnable
In LangChain, many components are designed to be Runnable, meaning you can call them with .invoke(), chain them with |, or deploy them via APIs. **A component in the Chain must be a runnable** .

Here is the Runnable list: 

- PromptTemplate - Renders a prompt using input variables

- LLM - Calls the language model with the prompt

- OutputParser - Parses raw model output (e.g., to list, dict, or clean string)

- Retriever - Retrieves relevant documents from a vector store

- RunnableLambda - Wraps a Python function (like lambda x: x.upper())

- RunnableParallel - Runs multiple Runnables in parallel and returns results

- RunnablePassthrough - Just returns the input (like identity function)

- RunnableBranch - Routes input to one of multiple branches based on logic 

- RunnableWithFallbacks - 	Runs a primary Runnable, falls back to backups if it fails

- create_retrieval_chain() - Chains a retriever + LLM + prompt into one pipeline

- create_stuff_documents_chain() - Converts docs + prompt into a final LLM query

- Custom @Runnable - A decorator that can make your class and functions runnable. 


```python
chain = prompt | llm | parser
response = chain.invoke({"topic": "LangChain"})

```

In [ ]:
## Using RunnableLambda
from langchain_core.runnables import RunnableLambda

uppercase = RunnableLambda(lambda x: x.upper())
print(uppercase.invoke("hello"))  # Output: "HELLO"


In [ ]:
## Create a Custom Runnable Class
from langchain_core.runnables import Runnable

class MyCustomRunnable(Runnable):
    def invoke(self, input, config=None):
        return input[::-1]  # reverses string

MyCustomRunnable().invoke("langchain")  # → "niahcnagnal"


In [ ]:
## Create a Custom Runnable by using decorator @runnable

from langchain_core.runnables import runnable

@runnable
def shout_reverse(text: str) -> str:
    return text[::-1].upper()

print(shout_reverse.invoke("hello"))  # Output: "OLLEH"

In [12]:
## Using RunnableParallel

from langchain_core.runnables import RunnableLambda, RunnableParallel

# Define individual runnables
reverse = RunnableLambda(lambda x: x[::-1])
uppercase = RunnableLambda(lambda x: x.upper())
lowercase = RunnableLambda(lambda x: x.lower())

# Combine using RunnableParallel
parallel = RunnableParallel({
    "reversed": reverse,
    "upper": uppercase,
    "lower": lowercase
})

# Invoke with a single input
result = parallel.invoke("LangChain")
print(result)




{'reversed': 'niahCgnaL', 'upper': 'LANGCHAIN', 'lower': 'langchain'}


In [19]:
## using RunnableWithFallbacks

from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks

# Primary: will raise an error
primary1 = RunnableLambda(lambda x: 1 / 0)  # Will fail

# Fallback: returns a default string
fallback = RunnableLambda(lambda x: "Fallback response")


# Combine them
resilient_runnable = RunnableWithFallbacks(
    runnable=primary,
    fallbacks=[fallback]
)

print(resilient_runnable.invoke("Test"))


Fallback response


In [ ]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("Explain {concept} in simple terms.")
# llm = Ollama(model="gemma:2b", temperature=0.1)

from langchain_openai import ChatOpenAI
llm= ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4o")
parser = StrOutputParser()

from langchain_core.runnables import RunnableLambda

# Custom step: count words in the output
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}")

chain = prompt | llm | parser | count_words

result = chain.invoke({"concept": "machine learning"})
print(result)
